## Simple simulations with minibn

The module `minibn` implements basic methods for computing simulation runs with usual synchronous, fully-asynchronous, and (general) asynchronous update modes. For the two latter, the `seed` argument ensures the reproducibility of traces.
The implementation also enables defining custom update modes.

In [1]:
from colomoto import minibn
import pandas as pd # for displaying traces

In [2]:
bn = minibn.BooleanNetwork({
    "a": "c",
    "b": "!a",
    "c": "!b"
})

Run synchronous simulation from zero state, for at most 10 iterations:

In [3]:
pd.DataFrame(minibn.SyncRun(bn, bn.zero(), 10))

,a,b,c
0,0,0,0
1,0,1,1
2,1,1,0
3,0,0,0
4,0,1,1
5,1,1,0
6,0,0,0
7,0,1,1
8,1,1,0
9,0,0,0


Run fully-aynchronous simulation from zero state, for at most 10 iterations:

In [4]:
pd.DataFrame(minibn.FAsyncRun(bn, bn.zero(), 10, seed=23892830))

,a,b,c
0,0,0,0
1,0,0,1
2,0,1,1
3,0,1,0


Run (general) aynchronous simulation from zero state, for at most 10 iterations:

In [5]:
pd.DataFrame(minibn.GAsyncRun(bn, bn.zero(), 10, seed=12323892))

,a,b,c
0,0,0,0
1,0,1,1
2,0,1,0


We define a custom update mode by extending from the general asynchronous mode.
The method `select_for_update` is called to select which nodes to update from the given list of nodes which can change of value in the current state.

In this example, we implement a simple priority mecanisms which will select the nodes with the highest priority, and then use the general asynchronous mode, i.e., picking randomly a subset of them:

In [6]:
class PrioGAsyncRun(minibn.GAsyncRun):
    def __init__(self, *args, priorities=[], **kwargs):
        super().__init__(*args, **kwargs)
        self.priorities = priorities
    def select_for_update(self, nodes):
        # select nodes by priority order
        for m in self.priorities:
            mn = set(m).intersection(nodes)
            if mn:
                nodes = list(mn)
                break
        # use parent class to select the nodes to update
        return super().select_for_update(nodes)

In [7]:
priorities = [{"c"},{"b"},{"a"}]
pd.DataFrame(PrioGAsyncRun(bn, bn.zero(), 10, priorities=priorities, seed=12323892))

,a,b,c
0,0,0,0
1,0,0,1
2,0,1,1
3,0,1,0
